In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget.exp import autotune_membrane
from voltagebudget.exp import autotune_w
from voltagebudget.exp import autotune_V_osc
from voltagebudget.util import read_modes
from voltagebudget.util import read_stim

from fakespikes import util as fsutil

# Tune oscillation (testing)

In [2]:
mode = 'regular'
N = 2
t = 0.3
E = 0.225
d = -6e-3

stim = "../data/stim1.csv"
stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

A_0 = 0.1e-9
A_max = 1e-9
phi_0 = 0
f = 8
solutions = autotune_V_osc(N, t, E, d, ns, ts, 
                           A_0=A_0, A_max=A_max, phi_0=phi_0, f=f, 
                           verbose=True)       

>>> Optimizing neuron 1/2.
>>> (A 0.000000000100, phi 0.000)  ->  (V_free 0.008, V_osc 0.012 loss -0.00400)
>>> (A 0.000000000100, phi 0.000)  ->  (V_free 0.008, V_osc 0.012 loss -0.00400)
>>> (A 0.000000000100, phi 0.000)  ->  (V_free 0.008, V_osc 0.012 loss -0.00400)
>>> Optimizing neuron 2/2.
>>> (A 0.000000000100, phi 0.000)  ->  (V_free 0.008, V_osc 0.012 loss -0.00400)
>>> (A 0.000000000100, phi 0.000)  ->  (V_free 0.008, V_osc 0.012 loss -0.00400)
>>> (A 0.000000000100, phi 0.000)  ->  (V_free 0.008, V_osc 0.012 loss -0.00400)


In [ ]:
solutions

In [ ]:
# Select neuron
n = 0

# Opt oscillations
A, phi = solutions[n].x
print("Optimal A {}, phi {}, f {}".format(A, phi, f))

# Other params
params, w_in, bias_in, sigma = read_modes(mode)

stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

# !
ns_y, ts_y, voltages_y = neurons.adex(
                                  N, t, 
                                  ns, ts,
                                  w_in=w_in, 
                                  bias_in=bias_in, 
                                  sigma=0,
                                  f=f,
                                  A=A,
                                  phi=phi,
                                  **params)

# -
times = voltages_y['times']
v = voltages_y['V_m'][n, :]

p = figure(plot_width=400, plot_height=200)
p.line(x=times, y=v, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Tune the bias 

In [ ]:
# Print options
util.get_mode_names()

In [ ]:
for mode in util.get_mode_names():
    print(">>> Tuning {}.".format(mode))
    
    params, _, bias_0, sigma_0 = util.read_modes(mode)
    sol = autotune_membrane(mode, bias_0, sigma_0, -65e-3, -2e-3)
    bias_x, sigma_x = sol.x
    
    np.savez("../data/{}_membrane_tuned".format(mode), bias=bias_x, sigma=sigma_x)

### Plot examples

In [ ]:
mode = 'adaption'

In [ ]:
params, _, _, _ = util.read_modes(mode)
sol = np.load("../data/{}_membrane_tuned.npz".format(mode))

bias_x = float(sol['bias'])
sigma_x = float(sol['sigma'])

print(bias_x, sigma_x)

# -
t = 1
ns_y, ts_y, budget = neurons.adex(1, t, 
                                  np.asarray([0]), np.asarray([0]), 
                                  w_max=0, 
                                  bias=bias_x, 
                                  sigma=sigma_x,
                                  f=0,
                                  **params)

# -
times = budget['times']
v = budget['V_m'][0, :]

p = figure(plot_width=400, plot_height=200)
p.line(x=times, y=v, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
params

- After plotting each to confirm everything looked OK, the tuned values were hand tranfered to the defulat json file

# Tune w

After entering the optimal bias/sigma into default json, I tuned `w_max`.

In [ ]:
for mode in util.get_mode_names():
    print(">>> Tuning {}.".format(mode))
    
    params, w_0, _, _ = util.read_modes(mode)
    sol = autotune_w(mode, w_0, 10, max_mult=1.5)
    w_x = sol.x
    print(w_x)
    np.savez("../data/{}_w_tuned".format(mode), w=w_x)

### Plot examples

In [ ]:
# Overall run time
t = 3

# Create frozen input spikes
stim_rate = 30
seed_stim = 1
k = 20
stim_onset = 0.1
stim_offset = t
dt = 1e-5

ns, ts = util.poisson_impulse(
    t,
    stim_onset,
    stim_offset - stim_onset,
    stim_rate,
    n=k,
    dt=dt,
    seed=seed_stim)

In [ ]:
mode = 'regular'

In [ ]:
params, _, bias, sigma = util.read_modes(mode)

sol = np.load("../data/{}_w_tuned.npz".format(mode))
w_x = float(sol['w'])
print(w_x)

# -
t = 1
N = 100
ns_y, ts_y, budget = neurons.adex(N, t, 
                                  ns, ts,
                                  w_max=w_x*1.3, 
                                  bias=bias, 
                                  sigma=sigma,
                                  f=0,
                                  **params)

# -
p = figure(plot_width=400, plot_height=200)
p.circle(ts_y, ns_y, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

p = figure(plot_width=400, plot_height=200)
for i in range(N):
    times = budget['times']
    v = budget['V_m'][i, :]

    p.line(x=times, y=v, color="black", alpha=0.1)
    p.xaxis.axis_label = 'Time (s)'
    p.yaxis.axis_label = 'Vm (volts)'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
show(p)